In [9]:
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd
import os # allows us to read the filenames in order to create the image_id collumn
import glob # allows you to import all images using a list comprehension
from skimage import morphology
from skimage.transform import rotate

In [10]:
masks = [plt.imread(file) for file in glob.glob("../data/example_segmentation/*.png")] # importing all images

In [11]:
image_ids = []

for root, dirs, files in os.walk("../data/example_image"):
    for name in files:
        if name[-3:] != "jpg":
            continue
        image_ids.append(name[0:-4])
        
image_ids = np.array(image_ids)


Areas = np.array([np.sum(mask) for mask in masks])

brush = morphology.disk(3)
Perimeters = np.array([np.sum(mask) - np.sum(morphology.binary_erosion(mask, brush)) for mask in masks])




In [ ]:
def get_assymetry(mask, rotation = 180):
    axes = 0
    mask = np.pad(mask, 2000) # adding empty space around the image so no information is lost on rotation
    length, width = mask.shape
    diffs = []
    
    while axes * rotation < 180:
        temp_mask = rotate(mask, axes * rotation)
        length_lesion = np.nonzero(np.sum(temp_mask, axis = 0))[0][-1] - np.nonzero(np.sum(temp_mask, axis = 0))[0][0]
        left_mask = temp_mask[0: width, 0:np.nonzero(np.sum(temp_mask, axis = 0))[0][0] + length_lesion//2]
        right_mask = temp_mask[0: width, np.nonzero(np.sum(temp_mask, axis = 0))[0][0] + length_lesion//2:length]
        diffs.append(np.abs(np.sum(left_mask)-np.sum(right_mask)))
        
        axes += 1
    
    diff = sum(diffs) / axes
    
    return diff / np.sum(mask)

assymetries = np.array([get_assymetry(mask, rotation = 15) for mask in masks])

In [ ]:
columns = ["image_id", "Area", "Perimeter", "Compactness", "Assymetry ratio"]
data = np.array([image_ids, Areas, Perimeters, np.round((4*np.pi*Areas)/Perimeters**2, 4), assymetries]).T
AreaPerimeterData = pd.DataFrame(data = data, columns = columns)
AreaPerimeterData